# Correlating Returns

In [43]:
import os
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
#import alpaca_trade_api as alpaca
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer


import alpaca_trade_api as trade_api
from alpaca_trade_api.rest import REST#, TimeFrame

import yfinance as yf

import datetime as dt


## Load API Keys from Environment Variables

In [44]:
# Load .env enviroment variables
load_dotenv()

# Set News API Key
newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = trade_api.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')
alpaca_api_key

'PKVRJK4WSE5AGXIQRK4N'

## Get AAPL Returns for Past Month

In [49]:
# Set the ticker
#ticker = "AAPL"

# Set timeframe to '1D'
#timeframe = "1D"

# Set current date and the date from one month ago using the ISO format
#current_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
#past_date = pd.Timestamp(datetime.now()- timedelta(30), tz="America/New_York").isoformat()

#print(past_date)

# Get 4 weeks worth of historical data for AAPL
#df = api.get_bars(
#    ticker,
#    TimeFrame.Day,
    #limit=None,
#    '2022-04-05',
#    '2022-05-10',
    #after=None,
    #until=None,
#    adjustment='raw'
#).df

#yearly = api.get_bars(tickers, TimeFrame.Day, "2021-03-20", "2022-03-20", adjustment='raw').df

# Display data
#df.head()


# Set the Tesla and Coca-Cola tickers
ticker = ["AAPL"]

# Set timeframe to "1Day"
timeframe = "1Day"

# Set start and end datetimes of 1 year, between now and 365 days ago.
start_date = pd.Timestamp("2022-04-11", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-05-11", tz="America/New_York").isoformat()

# Get 1 year's worth of historical data for Tesla and Coca-Cola
#df = api.get_bars(
#    ticker,
#    timeframe,
#    limit=None,
#    start=past_date,
#    end=current_date,
    #after=None,
    #until=None,
#).df









In [68]:
start = dt.datetime.today()-dt.timedelta(35)
end = dt.datetime.today()
ticker = 'AAPL'
df = yf.download(ticker, start, end)
df.tail()


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-05-05,163.850006,164.080002,154.949997,156.770004,156.540009,130525300
2022-05-06,156.009995,159.440002,154.179993,157.279999,157.279999,116055700
2022-05-09,154.929993,155.830002,151.490005,152.059998,152.059998,131577900
2022-05-10,155.520004,156.740005,152.929993,154.509995,154.509995,115366700
2022-05-11,153.500000,155.449997,145.809998,146.500000,146.500000,141075326


In [69]:
# Drop Outer Table Level
#df = df.droplevel(axis=1, level=0)
#print(df)
# Use the drop function to drop extra columns
df = df.drop(["Open", "High", "Low", "Volume","Adj Close"],axis=1)

# Since this is daily data, we can keep only the date (remove the time) component of the data
#df.index = df.index.date

# Display sample data
df.head()
df.to_excel('AAPLRaw.xlsx')

In [101]:
# Use the `pct_change` function to calculate daily returns of AAPL
aapl_returns = df.pct_change(3).shift(-3).dropna()

# Display sample data

aapl_returns.to_excel('aapl.xlsx')
aapl_returns.tail()

,Close
Date,
2022-05-02,-0.007534
2022-05-03,-0.013795
2022-05-04,-0.084086
2022-05-05,-0.014416
2022-05-06,-0.068540


In [50]:
# Use newsapi client to get most relevant 20 headlines per day in the past month
def get_headlines(keyword):
    all_headlines = []
    all_dates = []    
    date = datetime.strptime(current_date[:10], "%Y-%m-%d")
    end_date = datetime.strptime(past_date[:10], "%Y-%m-%d")
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > end_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=str(date)[:10],
            to=str(date)[:10],
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
    return all_headlines, all_dates

Note: Be aware that running the 3 requests below will only work once within a 24 hour period due to the request limits imposed by the API provider.

In [51]:
# Get first topic
aapl_headlines, dates = get_headlines("aapl")

Fetching news about 'aapl'
******************************
retrieving news from: 2022-05-11 00:00:00
retrieving news from: 2022-05-10 00:00:00
retrieving news from: 2022-05-09 00:00:00
retrieving news from: 2022-05-08 00:00:00
retrieving news from: 2022-05-07 00:00:00
retrieving news from: 2022-05-06 00:00:00
retrieving news from: 2022-05-05 00:00:00
retrieving news from: 2022-05-04 00:00:00
retrieving news from: 2022-05-03 00:00:00
retrieving news from: 2022-05-02 00:00:00
retrieving news from: 2022-05-01 00:00:00
retrieving news from: 2022-04-30 00:00:00
retrieving news from: 2022-04-29 00:00:00
retrieving news from: 2022-04-28 00:00:00
retrieving news from: 2022-04-27 00:00:00
retrieving news from: 2022-04-26 00:00:00
retrieving news from: 2022-04-25 00:00:00
retrieving news from: 2022-04-24 00:00:00
retrieving news from: 2022-04-23 00:00:00
retrieving news from: 2022-04-22 00:00:00
retrieving news from: 2022-04-21 00:00:00
retrieving news from: 2022-04-20 00:00:00
retrieving news fr

In [54]:
# Get second topic
#print(aapl_headlines)
trade_headlines, _ = get_headlines("trade")

Fetching news about 'trade'
******************************
retrieving news from: 2022-05-11 00:00:00
retrieving news from: 2022-05-10 00:00:00
retrieving news from: 2022-05-09 00:00:00
retrieving news from: 2022-05-08 00:00:00
retrieving news from: 2022-05-07 00:00:00
retrieving news from: 2022-05-06 00:00:00
retrieving news from: 2022-05-05 00:00:00
retrieving news from: 2022-05-04 00:00:00
retrieving news from: 2022-05-03 00:00:00
retrieving news from: 2022-05-02 00:00:00
retrieving news from: 2022-05-01 00:00:00
retrieving news from: 2022-04-30 00:00:00
retrieving news from: 2022-04-29 00:00:00
retrieving news from: 2022-04-28 00:00:00
retrieving news from: 2022-04-27 00:00:00
retrieving news from: 2022-04-26 00:00:00
retrieving news from: 2022-04-25 00:00:00
retrieving news from: 2022-04-24 00:00:00
retrieving news from: 2022-04-23 00:00:00
retrieving news from: 2022-04-22 00:00:00
retrieving news from: 2022-04-21 00:00:00
retrieving news from: 2022-04-20 00:00:00
retrieving news f

In [52]:
# Get third topic
economy_headlines, _ = get_headlines("economy")


Fetching news about 'economy'
******************************
retrieving news from: 2022-05-11 00:00:00
retrieving news from: 2022-05-10 00:00:00
retrieving news from: 2022-05-09 00:00:00
retrieving news from: 2022-05-08 00:00:00
retrieving news from: 2022-05-07 00:00:00
retrieving news from: 2022-05-06 00:00:00
retrieving news from: 2022-05-05 00:00:00
retrieving news from: 2022-05-04 00:00:00
retrieving news from: 2022-05-03 00:00:00
retrieving news from: 2022-05-02 00:00:00
retrieving news from: 2022-05-01 00:00:00
retrieving news from: 2022-04-30 00:00:00
retrieving news from: 2022-04-29 00:00:00
retrieving news from: 2022-04-28 00:00:00
retrieving news from: 2022-04-27 00:00:00
retrieving news from: 2022-04-26 00:00:00
retrieving news from: 2022-04-25 00:00:00
retrieving news from: 2022-04-24 00:00:00
retrieving news from: 2022-04-23 00:00:00
retrieving news from: 2022-04-22 00:00:00
retrieving news from: 2022-04-21 00:00:00
retrieving news from: 2022-04-20 00:00:00
retrieving news

In [53]:
# Instantiate SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [55]:
# Create function that computes average compound sentiment of headlines for each day
def headline_sentiment_summarizer_avg(headlines):
    sentiment = []
    for day in headlines:
        day_score = []
        for h in day:
            if h == None:
                continue
            else:
                day_score.append(sid.polarity_scores(h)["compound"])
        sentiment.append(sum(day_score) / len(day_score))
    return sentiment

In [56]:
# Get averages of each topics sentiment
aapl_avg = headline_sentiment_summarizer_avg(aapl_headlines)
trade_avg = headline_sentiment_summarizer_avg(trade_headlines)
economy_avg = headline_sentiment_summarizer_avg(economy_headlines)


In [102]:
# Combine Sentiment Averages into DataFrame
topic_sentiments = pd.DataFrame(
    {
        "aapl_avg": aapl_avg,
        "trade_avg": trade_avg,
        "economy_avg": economy_avg,
    }
)
topic_sentiments

#aapl_returns

,aapl_avg,trade_avg,economy_avg
0,0.076740,-0.027030,-0.122380
1,0.067515,0.024040,-0.218485
2,-0.014885,-0.018310,-0.020650
3,0.095914,0.066260,-0.035550
4,0.040829,-0.080905,-0.102625
5,0.041805,-0.023620,-0.149135
6,-0.014495,-0.049840,-0.074235
7,0.006645,-0.145065,-0.248965
8,0.087655,0.045800,-0.019290
9,0.023245,-0.120450,-0.026265


In [103]:
# Set the index value of the sentiment averages DataFrame to be the series of dates.
topic_sentiments.index = pd.to_datetime(dates)
#topic_sentiments

In [104]:
# Merge with AAPL returns
topic_sentiments = aapl_returns.join(topic_sentiments).dropna(how="any")

# Display data
#display(topic_sentiments)
topic_sentiments.to_excel("topicSent.xlsx")
topic_sentiments

,Close,aapl_avg,trade_avg,economy_avg
Date,,,,
2022-04-12,-0.015448,0.079215,0.018700,-0.004915
2022-04-13,-0.017606,0.105520,0.083810,-0.111880
2022-04-14,0.011737,0.140195,-0.064265,-0.040020
2022-04-18,0.008178,0.108115,-0.051430,-0.085290
2022-04-19,-0.033513,0.130300,0.040045,-0.159075
2022-04-20,-0.026012,-0.042135,0.019670,-0.028255
2022-04-21,-0.057806,0.075545,0.015255,-0.103115
2022-04-22,-0.032264,0.122855,-0.019990,0.020000
2022-04-25,0.004666,-0.007550,-0.022935,-0.006745


In [105]:
# Correlate the headlines' sentiment to returns
topic_sentiments.corr().style.background_gradient()

,Close,aapl_avg,trade_avg,economy_avg
Close,1.000000,-0.232454,-0.095465,0.455268
aapl_avg,-0.232454,1.000000,0.136174,0.060726
trade_avg,-0.095465,0.136174,1.000000,0.233755
economy_avg,0.455268,0.060726,0.233755,1.000000
